In [50]:
# Imports the function for calculating Cohen's kappa score 
from sklearn.metrics import cohen_kappa_score

In [51]:
import pandas as pd
import re
import numpy as np

In [84]:
def text_processing(filename):
    ''' 
    Opens the files created by prompting Llama3 through Powershell, and finds all instances of brackets, [],
    and adds these to a list.
    Then runs through each element in the list, which is now a string of numbers, and splits by comma
    and removes whitespace.
    `filename`: a textfile with output from prompting Llama3
    '''
    with open(filename, 'r', encoding='utf-16le') as infile:
        text = infile.read()
    
    bracket_list = re.findall(r'\[(.*?)\]', text)

    split_num_values = np.zeros((len(bracket_list),11))
    for i in range(len(bracket_list)):
        comma_sp = bracket_list[i].split(',')       # splits by comma
        x = np.zeros(11)
        for j in range(len(comma_sp)):
            x[j] = comma_sp[j].strip(' ')
        split_num_values[i] = x

    return split_num_values

### Creating instances for each file

Reading the pickle-file previosly created in as a dataframe

In [104]:
df = pd.read_pickle('df.pkl')

Converting this dataframe to an array to compare with the other values

In [105]:
ground_truth = df.iloc[0:, 2:-1].to_numpy()

In [58]:
#run_1 = text_processing('llama_output/output_drexl_1.txt')
run_2 = text_processing(r'C:\Users\maria\OneDrive - Universitetet i Oslo\VÅR2024\Textembedding\Drexl\llama_output\output_drexl_2.txt')
run_3 = text_processing(r'C:\Users\maria\OneDrive - Universitetet i Oslo\VÅR2024\Textembedding\Drexl\llama_output\output_drexl_3.txt')

### Cohen's Kappa score for Llama3

In [85]:
def nested_cohen(y1, y2):
    ''' 
    Computes Cohen's Kappa Score for a nested array by row and by column(category).
    `y1`: nested array
    `y2`: nested array
    '''
    # by row
    cohen_by_row = np.zeros(len(y1))
    for i in range(len(y1)):
        cohen_by_row[i] = cohen_kappa_score(y1[i],y2[i])

    # by column/category
    cohen_by_column = np.zeros(len(y1[0]))
    for i in range(len(y1[0])):
        y1_col = y1[:,i]
        y2_col = y2[:,i]
        cohen_by_column[i] = cohen_kappa_score(y1_col,y2_col)
        
    return cohen_by_row, cohen_by_column

In [109]:
a, b = nested_cohen(run_2, run_3)
print(a)
print(np.mean(a))
print(np.max(a), np.min(a))
print(np.var(a))
print(np.std(a))

print('b')
print(b)
print(np.mean(b))
print(np.std(b))

[0.26666667 0.09836066 0.81967213 ... 0.37735849 0.21428571 0.21428571]
0.338216959184075
1.0 -0.8333333333333335
0.10616291536361791
0.32582651114299754
b
[0.2897278  0.19275779 0.09860936 0.40400533 0.15456178 0.13627841
 0.1160208  0.12020517 0.10966349 0.13010518 0.12344765]
0.1704893419990963
0.08988534350428326


In [107]:
a, b = nested_cohen(ground_truth, run_2)
print(a)
print(np.mean(a))
print(np.max(a), np.min(a))
print(np.var(a))

print('b')
print(b)
print(np.mean(b))

[-0.22222222  0.47619048 -0.51724138 ...  0.42105263  0.02941176
  0.29787234]
0.13801863874833842
1.0 -0.7741935483870968
0.0799816678412158
b
[0.00962143 0.10594643 0.02325453 0.33332418 0.07743069 0.01397804
 0.05299897 0.07762211 0.0439125  0.1083958  0.06303668]
0.08268375910752898


### Cohen's Kappa Score for Centroid Method

In [10]:
print(f'{chr(954)}: Ground truth and Centroid Method = {cohen_kappa_score(ground_truth, embed_run)}')

κ: Ground truth and Centroid Method = 0.775840597758406


#### Excluded list

In [11]:
ground_t_c = ground_truth.copy()
embed_ex = embed_converter('embed_news_excluded.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'embed_news_excluded.csv'

Removes the "unknown" values, ie. the values where the greatest value in the distribution from the centroid method was less than 0.50, from both the ground truth and the list from the embeddings to get comparable data:

In [ ]:
i = 0
while i < len(embed_ex):
    if embed_ex[i] == 'unknown':
        embed_ex.pop(i)
        ground_t_c.pop(i)
    i += 1

In [ ]:
cohen_kappa_score(ground_t_c, embed_ex)

0.8212882389320273

### Cohen's Kappa Score for Llama3

In [ ]:
run_list = [ground_truth, llama_run1, llama_run2, llama_run3]
run_name = ['Ground truth', 'Run 1', 'Run 2', 'Run 3']

for i in range(0,len(run_list)):
    for j in range(i+1, len(run_list)):
        print(f'{chr(954)}: {run_name[i]} and {run_name[j]} = {cohen_kappa_score(run_list[i],run_list[j])}')

κ: Ground truth and Run 1 = 0.8748354085622594
κ: Ground truth and Run 2 = 0.869899822863605
κ: Ground truth and Run 3 = 0.8650661176023748
κ: Run 1 and Run 2 = 0.9550195664885774
κ: Run 1 and Run 3 = 0.9525706697021438
κ: Run 2 and Run 3 = 0.9650587013816788
